# Store kitti sequence

Create a sequence

In [1]:
# when you change a setting you have to restart the kernel (settings are only loaded once)
import os
FRAMECOUNT = 1101

import sys
sys.path.append('..')
from src.kitti import *
%matplotlib inline
import numpy as np
import glob
import urbg2o
import cv2


IMAGE_SIZE = 400

def load(file):
    keyframeids, frameids, poses = load_keyframes(file)
    return keyframeids, frameids, poses.reshape(poses.shape[0], 4, 4)

def load_csv(file):
    poses = np.genfromtxt(file, delimiter= " ")
    poses = np.hstack([poses, np.repeat(np.array([[0,0,0,1.0]]), len(poses), axis=0) ])
    return poses.reshape(poses.shape[0], 4, 4)

def transform(poses):
    points = np.repeat(np.array([[0,0,0,1.0]]), len(poses), axis=0) 
    for i in range(1, len(poses)):
        points[:i] = np.dot(points[:i], poses[i].T)
    return points
        
def transform2(poses):
    points = poses[:,:,3]
    return points
    
def plot_trajectory(points):
    # remove y coordinate (we only use x and z to plot a view from above)
    points = points[:, [0,2,3]]
    # compute min and max
    pmin = points.min(axis=0)
    pmax = points.max(axis=0)
    maxxy = max(pmax[0] - pmin[0], pmax[1] - pmin[1])

    t = np.eye(3, dtype=np.float64)
    t[0,0] = IMAGE_SIZE / maxxy
    t[1,1] = -IMAGE_SIZE / maxxy
    t[0,2] = -pmin[0] * IMAGE_SIZE / maxxy
    t[1,2] = IMAGE_SIZE / maxxy + pmin[1] / maxxy + 100

    return np.dot(points, t.T)
    

def plot_trajectory2(points):
    # remove y coordinate (we only use x and z to plot a view from above)
    points = points[:, [0,2,3]]
    # compute min and max
    pmin = points.min(axis=0)
    pmax = points.max(axis=0)
    maxxy = max(pmax[0] - pmin[0], pmax[1] - pmin[1]) * 1.01

    t = np.eye(3, dtype=np.float64)
    t[0,0] = IMAGE_SIZE / maxxy
    t[1,1] = -IMAGE_SIZE / maxxy
    t[0,2] = -pmin[0] * IMAGE_SIZE / maxxy
    t[1,2] = IMAGE_SIZE / maxxy + pmin[1] / maxxy + 100
    return np.dot(points, t.T)
    
def plot_trajectory3(points):
    coords = []
    minx = min([p[0] for p in points])
    maxx = max([p[0] for p in points])
    miny = min([p[2] for p in points])
    maxy = max([p[2] for p in points])
    maxxy = max(maxx - minx, maxy - miny)* 1.01

    for p in points:
        #print(p)
        x = IMAGE_SIZE * (p[0] - minx) / (maxxy)
        z = IMAGE_SIZE - IMAGE_SIZE * (p[2] - miny) / (maxxy)
        coords.append([x, z])
    return coords

def new_image():
    img = np.zeros((IMAGE_SIZE, IMAGE_SIZE, 3), np.uint8)
    img[:] = 255 #or img.fill(255)
    return img

def draw(coords, img, color = (0,0,0)):
    # Draw a black line with thickness of 2 px
    for i in range(1,len(coords)):
        cv2.line(img, (int(coords[i - 1][0]), int(coords[i - 1][1])), (int(coords[i][0]), int(coords[i][1])), color, 2)
    return img

Read the first 10 frames from KITTI sequence 00.

In [2]:
%%time

frame = []
for filename in sorted(glob.glob(get_leftdir() + '/*')):
    left_frame = Frame(filename)
    frame.append(left_frame)

CPU times: user 20.8 ms, sys: 2.59 ms, total: 23.4 ms
Wall time: 22.5 ms


Add the frames to a sequence. By adjusting the [:] you can use only a slice of the list of frames.

In [4]:
seq = Sequence()
for f in ProgressBar()(frame[:10]):
    seq.add_keyframe(f,run_ba = False)

pose_frame(frame[1060],frame[1])

for obs in frame[1060].get_observations():
    # obs matches previous keyframe
    if obs.has_mappoint():
        obs.check_mappoint()
        obs.register_mappoint()
        
                
    

 20% (2 of 10) |#####                     | Elapsed Time: 0:00:00 ETA:  0:00:00

connected mappoints  239
connected mappoints  268


 40% (4 of 10) |##########                | Elapsed Time: 0:00:00 ETA:  0:00:00

connected mappoints  269
connected mappoints  276


 60% (6 of 10) |###############           | Elapsed Time: 0:00:00 ETA:  0:00:00

connected mappoints  273
connected mappoints  281


 80% (8 of 10) |####################      | Elapsed Time: 0:00:01 ETA:  0:00:00

connected mappoints  285
connected mappoints  306


100% (10 of 10) |#########################| Elapsed Time: 0:00:01 Time: 0:00:01


connected mappoints  280
connected mappoints  270


Example of how to save a set of covisible keyframes, fixed keyframes, mappoints and edges to use for local bundle adjustment

In [5]:
seq.keyframes[0].get_world_pose()

array([[ 1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  1.]])

In [7]:
seq.keyframes[9].get_world_pose()

array([[  9.96477973e-01,  -9.80315399e-04,   8.38491973e-02,
         -3.46852245e-01],
       [ -8.51464586e-04,   9.99761824e-01,   2.18075745e-02,
         -3.43329106e-02],
       [ -8.38506048e-02,  -2.18021622e-02,   9.96239801e-01,
         -8.35128334e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          1.00000000e+00]])

In [8]:
mp1 = [o for o in seq.keyframes[9].get_observations() if o.mappoint is not None]
[o.get_affine_coords() for o in mp1][:5]

[array([-12.3126001 ,  -0.49462031,  31.70205844,   1.        ]),
 array([ -6.19745632,   1.00224478,  17.66536816,   1.        ]),
 array([  0.90324792,  -3.72614726,  16.31124986,   1.        ]),
 array([ 2.23025038,  1.56849016,  8.4913545 ,  1.        ]),
 array([ -4.82020329,   1.63550305,  12.02331236,   1.        ])]

In [9]:
mp1 = [o for o in seq.keyframes[9].get_observations() if o.mappoint is not None]
[o.get_world_coords() for o in mp1][:5]

[array([-15.28170919,  -1.32169702,  38.88937715,   1.        ]),
 array([ -8.01239266,   0.47484698,  25.45086051,   1.        ]),
 array([ -0.8191276 ,  -4.22985709,  24.59410756,   1.        ]),
 array([  1.15439589,   1.23270902,  17.03034785,   1.        ]),
 array([ -6.16743976,   1.2296133 ,  19.95931135,   1.        ])]

In [5]:
mp1 = [o.mappoint for o in frame[1].get_observations() if o.mappoint is not None]
[len(m.observations) for m in mp1]

[1,
 3,
 3,
 3,
 4,
 2,
 2,
 5,
 3,
 3,
 1,
 1,
 3,
 4,
 2,
 2,
 4,
 1,
 12,
 12,
 7,
 2,
 4,
 1,
 3,
 1,
 2,
 5,
 5,
 4,
 2,
 6,
 1,
 1,
 1,
 1,
 3,
 1,
 8,
 2,
 4,
 1,
 5,
 9,
 6,
 2,
 2,
 6,
 6,
 3,
 9,
 1,
 1,
 4,
 2,
 7,
 2,
 1,
 2,
 3,
 2,
 8,
 2,
 1,
 3,
 4,
 4,
 2,
 2,
 3,
 4,
 1,
 3,
 2,
 1,
 5,
 10,
 1,
 5,
 8,
 4,
 3,
 4,
 3,
 2,
 3,
 1,
 3,
 3,
 4,
 3,
 1,
 8,
 3,
 13,
 3,
 2,
 2,
 6,
 2,
 5,
 2,
 3,
 3,
 8,
 1,
 2,
 2,
 2,
 2,
 2,
 7,
 1,
 2,
 1,
 5,
 8,
 4,
 6,
 2,
 3,
 9,
 4,
 3,
 6,
 4,
 7,
 4,
 4,
 5,
 3,
 3,
 1,
 6,
 1,
 5,
 3,
 3,
 3,
 6,
 3,
 3,
 2,
 6,
 3,
 3,
 5,
 2,
 2,
 1,
 3,
 1,
 4,
 5,
 3,
 2,
 4,
 1,
 1,
 4,
 2,
 3,
 1,
 3,
 2,
 2,
 2,
 6,
 1,
 3,
 4,
 3,
 4,
 3,
 2,
 6,
 2,
 3,
 5,
 2,
 5,
 13,
 2,
 2,
 1,
 3,
 2,
 1,
 3,
 1,
 1,
 4,
 6,
 1,
 2,
 6,
 2,
 3,
 2,
 1,
 16,
 2,
 1,
 2,
 1,
 3,
 1,
 2,
 2,
 1,
 9,
 3,
 9,
 1,
 1]

In [10]:
[ f.cumulative_pose for f in seq.keyframes[:5]]


AttributeError: 'Frame' object has no attribute 'cumulative_pose'

In [ ]:

print('Keyframes count\n', (len(seq.keyframes)), '\n')

def keyframes_to_np(keyframes):
    return np.hstack([ [ [ kf.keyframeid] for kf in keyframes ], [ [kf.frameid] for kf in keyframes ], [ kf.cumulative_pose.flatten() for kf in keyframes ] ])

count = 10
poses = np.array([ np.eye(4) for f in seq.keyframes[:count] ])
observations = [ m.get_last_observation() for m in get_mappoints( seq.keyframes[:count])
obs = [o.get_affine_coords() for o in observations ]
                
for i in range(count)):
    pose = seq.keyframes[i].get_pose()
    for j in range(count-i+1):
        pose[j] = np.dot( pose, poses[j] )
        for k, o in enumerate(observations):
            if o.frame == seq.keyframes[i]:
                obs[k] = np.dot(obs[k], pose)

for i in range(len(seq.keyframes)):
    
    f = seq.keyframes[i]
    pose = f.get_pose()
    
    if i > 0:
        f.cumulative_pose = np.dot(seq.keyframes[i-1].cumulative_pose, pose)
    else:
        f.cumulative_pose = pose

cv_keyframes = seq.keyframes
mappoints = get_mappoints(cv_keyframes)
f_keyframes = get_fixed_keyframes(mappoints, cv_keyframes)[:0]

cv_keyframes_np = keyframes_to_np(cv_keyframes)
f_keyframes_np = keyframes_to_np(f_keyframes)
mappoints_np = mappoints_to_np(mappoints)
links_np = links_to_np(mappoints)


f_keyframes = np.asfortranarray(f_keyframes_np)
cv_keyframes_BA = np.asfortranarray(cv_keyframes_np)
mappoints = np.asfortranarray(mappoints_np)
links = np.array(links_np, order='f')

# np.set_printoptions(threshold=np.nan)

print('mappoints count  \n', (len(mappoints[:]), '\n') )
print('mappoints \n', (mappoints[:]), '\n')


print('links count  \n', (len(links[:]), '\n') )
print('links \n', (links[:]), '\n')


print('Keyframes \n', (cv_keyframes), '\n')
print('Keyframes numpy \n', (cv_keyframes_np[:, 2 :]), '\n')
print('mappoints len \n', len(mappoints_np), '\n')
print('Fixed Keyframes \n',f_keyframes[:], '\n')
# print( keyframes_to_np( [seq.keyframes[1]]  )[:, 2 :] , '\n')


print('Keyframes before BA \n', (cv_keyframes_BA[: , 2:]), '\n')
print(len(cv_keyframes_BA), '\n')

result =  urbg2o.fullBundleAdjustment(cv_keyframes_BA, mappoints, links)

print('Keyframes after BA \n', (b[:, 2 :]), '\n')
print(len(cv_keyframes_BA), '\n')




# print(cv_keyframes[:, 0 :], '\n')

np_matrix_values =  cv_keyframes_np[:,2 :]


# draw seq.keyframes

# array_value = []
# for i in range(0, len(keyframes_to_np(seq.keyframes)[:,2 :]) ):
#     array_value.append(  np.split(keyframes_to_np(seq.keyframes)[:,2 :][i], 4 )  )
# np_array_value = np.array(array_value)

# points = transform( np_array_value )
# coords = plot_trajectory3(points)
# p = draw(coords, new_image())
# show(p)



#old points 
# array_value = []
# for i in range(0, len(np_matrix_values) ):
#     array_value.append(  np.split(np_matrix_values[i], 4 )  )
# np_array_value = np.array(array_value)

# points = transform( np_array_value )
# coords = plot_trajectory3(points)
# p = draw(coords, new_image())
# show(p)

# #new points

# start_array_value = []
# for i in range(0, len(cv_keyframes_BA[:,2 :] ) ):
#     start_array_value.append(  np.split(cv_keyframes_BA[:,2 :][i], 4 )  )
# start_array_value = np.array(start_array_value)

# points_start = transform( start_array_value )
# coords_start = plot_trajectory3(points_start)
# p_start = draw(coords_start, new_image())
# show(p_start)


keyframes_np_MO = keyframes_to_np(seq.keyframes)

np.save('cv_keyframes', cv_keyframes_BA)
np.save('f_keyframes', f_keyframes_np)
np.save('mappoints', mappoints_np)
np.save('links', links_np)


Keyframes count
 1101 

mappoints count  
 (65544, '\n')
mappoints 
 [[  1.66491000e+05  -7.17758759e+01  -1.30994739e+01  -1.03454706e+02
    1.00000000e+00]
 [  1.83451000e+05  -6.20044465e+01  -1.81241300e+01  -2.78673742e+01
    1.00000000e+00]
 [  2.05995000e+05  -5.30001411e+00  -2.90369938e+01   1.12874555e+02
    1.00000000e+00]
 ..., 
 [  9.59330000e+04  -2.17988905e+02  -2.57260395e+01   2.56337899e+01
    1.00000000e+00]
 [  1.01643000e+05  -1.24986995e+02  -1.66435166e+00  -1.55935449e+02
    1.00000000e+00]
 [  1.46891000e+05  -1.55323769e+02  -1.10465143e+01   7.71703362e+01
    1.00000000e+00]] 

links count  
 (195129, '\n')
links 
 [[  1.66491000e+05   8.06000000e+02   8.21000000e+02   1.48000000e+02]
 [  1.66491000e+05   8.04000000e+02   8.04000000e+02   1.48000000e+02]
 [  1.66491000e+05   8.05000000e+02   8.12000000e+02   1.48000000e+02]
 ..., 
 [  1.46891000e+05   7.06000000e+02   9.37000000e+02   8.40000000e+01]
 [  1.46891000e+05   7.10000000e+02   9.37000000e+02

Keyframes after BA 
 [[  1.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   1.00000000e+00]
 [  9.99979101e-01  -3.93699186e-04   6.45316462e-03 ...,   0.00000000e+00
    0.00000000e+00   1.00000000e+00]
 [  9.99910769e-01  -1.18251883e-03   1.33062412e-02 ...,   0.00000000e+00
    0.00000000e+00   1.00000000e+00]
 ..., 
 [  9.97837211e-01  -3.85488063e-02   5.32436794e-02 ...,   0.00000000e+00
    0.00000000e+00   1.00000000e+00]
 [  9.97825330e-01  -3.89698781e-02   5.31597586e-02 ...,   0.00000000e+00
    0.00000000e+00   1.00000000e+00]
 [  9.97826254e-01  -3.90197916e-02   5.31057706e-02 ...,   0.00000000e+00
    0.00000000e+00   1.00000000e+00]] 

1101 



In [7]:


for i in range(len(seq.keyframes) - 1, 0, -1):
    f = seq.keyframes[i]
    prev = seq.keyframes[i-1]
    previnv = np.linalg.pinv(prev.cumulative_pose)
    pose = np.dot(previnv, f.cumulative_pose)
    print(i, f.get_pose(), '\n', pose)

1100 [[  9.99999997e-01  -4.87841094e-05  -5.32673355e-05  -3.60174281e-04]
 [  4.87853387e-05   9.99999999e-01   2.30766586e-05   1.88839815e-05]
 [  5.32662096e-05  -2.30792572e-05   9.99999998e-01   4.18430086e-03]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]] 
 [[  9.99999997e-01  -4.87841094e-05  -5.32673355e-05  -3.60174281e-04]
 [  4.87853387e-05   9.99999999e-01   2.30766586e-05   1.88839815e-05]
 [  5.32662096e-05  -2.30792572e-05   9.99999998e-01   4.18430086e-03]
 [  7.99389931e-17   9.41084927e-17  -4.91624616e-17   1.00000000e+00]]
1099 [[  9.99999889e-01  -4.58521071e-04  -1.10491580e-04   1.05198406e-03]
 [  4.58445339e-04   9.99999661e-01  -6.84457347e-04   2.11775952e-03]
 [  1.10805381e-04   6.84406616e-04   9.99999760e-01  -4.17261582e-04]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]] 
 [[  9.99999889e-01  -4.58521071e-04  -1.10491580e-04   1.05198406e-03]
 [  4.58445339e-04   9.99999661e-01  -6.84457347e-04   2.117

 [[  9.98517709e-01   2.61520745e-03   5.43649209e-02   2.81557877e-03]
 [ -2.74914692e-03   9.99993367e-01   2.38907329e-03   2.73179501e-03]
 [ -5.43583124e-02  -2.53498914e-03   9.98518276e-01  -5.00717683e-01]
 [ -4.36056448e-15   1.03066833e-15   5.73525586e-15   1.00000000e+00]]
904 [[ 0.99856214 -0.00192501  0.05357185  0.01012977]
 [ 0.00183237  0.99999674  0.00177831  0.00600686]
 [-0.0535751  -0.00167759  0.99856241 -0.49357423]
 [ 0.          0.          0.          1.        ]] 
 [[  9.98562142e-01  -1.92501067e-03   5.35718491e-02   1.01297744e-02]
 [  1.83237172e-03   9.99996740e-01   1.77831008e-03   6.00686316e-03]
 [ -5.35750977e-02  -1.67758958e-03   9.98562414e-01  -4.93574225e-01]
 [ -3.69278295e-15  -8.16674577e-15   6.92720252e-15   1.00000000e+00]]
903 [[  9.98603413e-01  -4.10860191e-03   5.26720232e-02   1.09248939e-02]
 [  4.06681483e-03   9.99991325e-01   9.00498875e-04   2.76451539e-03]
 [ -5.26752661e-02  -6.85033885e-04   9.98611460e-01  -5.01057437e-01]
 

 [[  9.99999833e-01  -5.26877929e-04   2.38289653e-04  -1.15884852e-03]
 [  5.26876320e-04   9.99999861e-01   6.81440284e-06  -2.13722200e-03]
 [ -2.38293210e-04  -6.68885251e-06   9.99999972e-01   7.95328950e-05]
 [ -7.66629167e-17  -1.26858617e-17  -1.19558499e-16   1.00000000e+00]]
690 [[  9.99999999e-01  -1.79139857e-05  -4.78531991e-05   1.10787336e-03]
 [  1.79029688e-05   9.99999973e-01  -2.30212794e-04   2.29059820e-03]
 [  4.78573219e-05   2.30211937e-04   9.99999972e-01  -5.66087644e-04]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]] 
 [[  9.99999999e-01  -1.79139857e-05  -4.78531991e-05   1.10787336e-03]
 [  1.79029688e-05   9.99999973e-01  -2.30212794e-04   2.29059820e-03]
 [  4.78573219e-05   2.30211937e-04   9.99999972e-01  -5.66087644e-04]
 [ -6.84408986e-17   1.29264163e-17  -1.05688823e-16   1.00000000e+00]]
689 [[  9.99999998e-01   6.30302407e-05   1.69096183e-05  -4.77204189e-04]
 [ -6.30308055e-05   9.99999997e-01   3.34070491e-05   1.511342

 [[  9.99999997e-01   2.72427539e-05   7.19177742e-05  -1.31787223e-03]
 [ -2.72317185e-05   9.99999988e-01  -1.53441878e-04   1.63610672e-03]
 [ -7.19219535e-05   1.53439919e-04   9.99999986e-01   3.47127847e-04]
 [ -6.31809907e-17  -1.00204455e-16  -1.18085164e-16   1.00000000e+00]]
674 [[  9.99999999e-01   4.80432476e-05   2.41547362e-05  -2.40909272e-04]
 [ -4.80418418e-05   9.99999997e-01  -5.81950153e-05   8.29753290e-04]
 [ -2.41575320e-05   5.81938547e-05   9.99999998e-01   5.58934526e-04]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]] 
 [[  9.99999999e-01   4.80432475e-05   2.41547362e-05  -2.40909273e-04]
 [ -4.80418418e-05   9.99999997e-01  -5.81950153e-05   8.29753290e-04]
 [ -2.41575320e-05   5.81938547e-05   9.99999998e-01   5.58934526e-04]
 [ -7.30470198e-17   6.51033596e-17  -1.13162483e-16   1.00000000e+00]]
673 [[  9.99999982e-01   1.84827174e-04  -4.81427431e-05   1.41826169e-03]
 [ -1.84820532e-04   9.99999973e-01   1.37945028e-04   2.094367

 [[  9.98634909e-01   7.03170389e-03   5.17578289e-02   1.76144170e-02]
 [ -7.11810182e-03   9.99973563e-01   1.48512747e-03   4.99525831e-03]
 [ -5.17460176e-02  -1.85151763e-03   9.98658561e-01  -4.85120993e-01]
 [  1.08248090e-17  -1.24497522e-15  -3.06624762e-17   1.00000000e+00]]
469 [[  9.98629610e-01   3.01880606e-03   5.22473870e-02   2.11201369e-02]
 [ -3.06286866e-03   9.99995018e-01   7.63297773e-04   3.57934994e-03]
 [ -5.22448224e-02  -9.22278642e-04   9.98633881e-01  -4.95420570e-01]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]] 
 [[  9.98629610e-01   3.01880606e-03   5.22473870e-02   2.11201369e-02]
 [ -3.06286866e-03   9.99995018e-01   7.63297773e-04   3.57934994e-03]
 [ -5.22448224e-02  -9.22278642e-04   9.98633881e-01  -4.95420570e-01]
 [ -4.61023219e-18  -2.28405427e-17   1.98744665e-17   1.00000000e+00]]
468 [[ 0.99869014 -0.00345054  0.05105002  0.03644176]
 [ 0.00361548  0.99998854 -0.00313896  0.00665856]
 [-0.0510386   0.00331942  0.998

270 [[  9.99986235e-01  -1.19337552e-04   5.24553218e-03   1.02039227e-02]
 [  1.20770806e-04   9.99999955e-01  -2.72917283e-04   1.36516425e-02]
 [ -5.24549938e-03   2.73547033e-04   9.99986205e-01  -7.85070432e-01]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]] 
 [[  9.99986235e-01  -1.19337552e-04   5.24553218e-03   1.02039227e-02]
 [  1.20770806e-04   9.99999955e-01  -2.72917283e-04   1.36516425e-02]
 [ -5.24549938e-03   2.73547033e-04   9.99986205e-01  -7.85070432e-01]
 [  3.96994975e-15   1.09880203e-16  -9.98393978e-16   1.00000000e+00]]
269 [[  9.99988052e-01  -9.00692156e-04   4.80458345e-03   8.88435233e-03]
 [  9.10201766e-04   9.99997631e-01  -1.97745948e-03   1.56826062e-02]
 [ -4.80279098e-03   1.98180899e-03   9.99986503e-01  -7.86094740e-01]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]] 
 [[  9.99988052e-01  -9.00692156e-04   4.80458345e-03   8.88435233e-03]
 [  9.10201766e-04   9.99997631e-01  -1.97745948e-03   1.56826

64 [[  9.99998984e-01   7.43458268e-04  -1.21599042e-03   1.09855242e-02]
 [ -7.45483092e-04   9.99998335e-01  -1.66555956e-03   1.79798082e-02]
 [  1.21475012e-03   1.66646437e-03   9.99997874e-01  -8.21187229e-01]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]] 
 [[  9.99998984e-01   7.43458268e-04  -1.21599042e-03   1.09855242e-02]
 [ -7.45483092e-04   9.99998335e-01  -1.66555956e-03   1.79798082e-02]
 [  1.21475012e-03   1.66646437e-03   9.99997874e-01  -8.21187229e-01]
 [ -4.96837457e-17  -6.71196440e-15  -2.72108276e-17   1.00000000e+00]]
63 [[ 0.999996    0.00260335 -0.00110146  0.01087062]
 [-0.00260537  0.99999491 -0.00183983  0.01022457]
 [ 0.00109666  0.00184269  0.9999977  -0.80718881]
 [ 0.          0.          0.          1.        ]] 
 [[  9.99996005e-01   2.60335126e-03  -1.10145749e-03   1.08706230e-02]
 [ -2.60537492e-03   9.99994914e-01  -1.83982878e-03   1.02245710e-02]
 [  1.09666217e-03   1.84269114e-03   9.99997701e-01  -8.07188807e-01]
 [